In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai import *
from fastai.vision import *

import warnings
warnings.filterwarnings("ignore")

%reload_ext autoreload
%autoreload 2
%matplotlib inline

np.random.seed(1)

In [ ]:
#Load images and apply some basic transformations and augmentations
trainCsvPath = '/kaggle/input/jpeg-melanoma-768x768/train.csv'
imgPath = '/kaggle/input/jpeg-melanoma-768x768/train'
df = pd.read_csv(trainCsvPath)
df['withJpg']=df['image_name']+'.jpg'
tfms = get_transforms(flip_vert=True, max_zoom=1.05, max_warp=0.)
data = ImageDataBunch.from_df(imgPath, df, fn_col='withJpg', label_col='benign_malignant', ds_tfms=tfms, size=224, bs=256)
data.normalize(imagenet_stats)

In [ ]:
fullSizeData = ImageDataBunch.from_df(imgPath, df, fn_col='withJpg', label_col='benign_malignant', ds_tfms=tfms, size=768, bs=32)

In [ ]:
#can use data instead of fullSizeData here when testing for faster processing
learn = create_cnn(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.data = fullSizeData

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
learn.freeze()

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
learn.model_dir = '../../../kaggle/working'

In [ ]:
learn.save('stage-1')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(1, max_lr=slice(3e-6,3e-4))

In [ ]:
learn.save('stage-2')

In [ ]:

testImageFilePaths = get_image_files('/kaggle/input/jpeg-melanoma-768x768/test')
    

In [ ]:
outputDf = pd.DataFrame(columns=['image_name','target'])

testDir = '/kaggle/input/jpeg-melanoma-768x768/test/'
i = 1
for filename in os.listdir(testDir):
    fullFilePath = testDir + filename
    classification,_,probabilities = learn.predict(open_image(fullFilePath))
    malignProbability = 1-probabilities[0]
    if str(classification) == 'malignant':
        malignProbability = probabilities[0]       
    formatted = float("{:.7f}".format(malignProbability))
    outputDf = outputDf.append({'image_name': filename[:-4],'target':formatted}, ignore_index=True)
    if i % 1000 == 0:
        print(f'{i}/{len(os.listdir(testDir))}')
    i+=1

In [ ]:
outputDf.to_csv('output.csv', index=False)